In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("SparkStreamWordCount")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

In [3]:
# pre-requistes

# we need a server running on port 9999, that provides text stream to spark
# spark stream is a client, that will connect ot a server running on port 9999

# terminal, 

# nc -lk 9999

In [3]:
# Structured Streaming 
# based on Data Frame/ internally there is RDD
# Micro Batch
# spark batch programming, we use spark.read
# in streaming, we use readStream
# socket is predefined word, represent data comes from TCP/IP socket
lineStreamDf = spark\
                .readStream\
                .format("socket")\
                .option("host", "localhost")\
                .option("port", 9999)\
                .load()

# now spark shall connect the server running on port 9999
# then it will wait the input from server
# user/developer can write something on server console and Press Enter key
# to send to spark / client

In [4]:
lineStreamDf.printSchema()
# show will not work for stream, it will cause error
#lineStreamDf.show() # good for batchs, not for stream

root
 |-- value: string (nullable = true)



In [5]:
from pyspark.sql.functions import split, explode, col
# split , split the string using space, convert line into word list
# explode project every element in list as element in stream [similar to flatMap]
# output is a stream of word
# Input "welcome to Kafka"
# split => ["welcome", "to", "Kafka"]
# explode makes every element as stream element
# welcome
# to
# Kafka
wordsDf = lineStreamDf.select(explode(split(lineStreamDf.value, " ")).alias("word"))
wordsDf.printSchema()

root
 |-- word: string (nullable = true)



In [6]:
# word count by group
# word  count
# Kafka  2..
wordCountDf = wordsDf.groupBy("word").count()

In [7]:
# print out on console 
# default batch, as soon possible , latest spark says 100 ms, expect 1 second delay
# check this cell or next cell
echoOnConsole = wordCountDf.writeStream.outputMode("complete").format("console").start()


In [7]:
# print out on console 
# Output: update
# only print if there is difference in output, not printing all results
# default batch, as soon possible , latest spark says 100 ms, expect 1 second delay
echoOnConsole = wordCountDf.writeStream.outputMode("update").format("console").start()
